In [4]:
import pandas as pd
from neo4j import GraphDatabase
import csv
from py2neo import Graph, Node, Relationship

JOB_LIST = ["Software Architect", "Business Analyst", "Data Scientist", "Game Development",
            "Database Administrator", "Data Engineer", "Data Analysts","Software Engineer", "Web Development",
            "Devops Engineer", "Network Engineer", "UI Designer", "Tester", "Mobile Developer", "Backend Developer", "Frontend Developer"]
DEFAULT_PATH = "/Users/nguyenvanviet/Work/Courses/DataGen/MyPaperDataSet/JobData/"
COMPETENCIES_LIST = ["Knowledge", "Platform", "Framework", "ProgrammingLanguage", "Tool"]
WEB_LIST = ["Indeed", "CareerBuilder", "Glint"]

## connect to neo4j (CareerInsightDW2 - for paper || ThesisDW for thesis) 

In [5]:
uri = "bolt://localhost:7687"  # Adjust the URI based on your Neo4j server configuration
username = "neo4j"
password = "12345678"

## using the neo4j library
# # Create a Neo4j driver instance
# driver = GraphDatabase.driver(uri, auth=(username, password))
# driver


graph = Graph(uri, auth=(username, password))

graph

Graph('bolt://localhost:7687')

## Remove the data    

In [6]:
def executeQuery(query):
    returnlist = []
    result = graph.run(query)
    for record in result:
        returnlist.append(record)
    
    return returnlist
            
    
def removeAllData():
    executeQuery("MATCH ()-[r]-() DELETE r;")
    executeQuery("MATCH (n) DELETE n;")
    print("Done! all data have been clear!")



In [7]:
#removeAllData()
executeQuery("MATCH(n) return n limit 25")

[Record({'n': Node('Knowledge', knowledge='CONCRETE STRUCTURE')}),
 Record({'n': Node('Knowledge', knowledge='STARE CASE')}),
 Record({'n': Node('Tool', tool='STRUCTURAL ANALYSIS AND DESIGN')}),
 Record({'n': Node('Tool', tool='STAAD PRO')}),
 Record({'n': Node('Tool', tool='STAAD.PRO')}),
 Record({'n': Node('Course', description='STAAD Pro stands for Structural Analysis and design. STAAD Pro v8i Software is widely used in analyzing and designing structures � buildings, bridges, towers, transportation, industrial and utility structures. Perform comprehensive analysis and design for any size or type of structure faster than ever before using the new STAAD.Pro V8i.Simplify your BIM workflow by using a physical model in STAAD.Pro that is automatically converted into the analytical model for your structural analysis. Share synchronized models with confidence for multi-discipline team collaboration and, most importantly, deliver safe, cost-effective designs. Introduction of STAAD Pro and St

## Import job data from csv to Neo4j
### Get the data from csv 

In [8]:
jobDataPath = DEFAULT_PATH + "job2024Clean.csv"
courseDataPath = DEFAULT_PATH + "courseClean.csv"

jobDataDf = pd.read_csv(jobDataPath)
courseDataDf = pd.read_csv(courseDataPath)
jobDataDf = jobDataDf.fillna("")
courseDataDf = courseDataDf.fillna("")

jobDataDf.head(5)
#courseDataDf.head(5) 

,companyName,location,timePost,website,jobType,jobCount,Knowledge,Platform,Framework,ProgrammingLanguage,Tool,date,month,year
0,AAA Minneapolis,"Saint Louis Park, MN 55416 (Wolfe Park area)",2024-01-14,Indeed,Database Administrator,3,",,BUSINESS INTELLIGENCE,DATA WAREHOUSE,DATA VI...",",,GOOGLE ANALYTICS,TABLEAU,DATA ANALYSIS,GOOGL...",",,SSIS,DATA ANALYSIS,DATA VISUALIZATION,DATABA...",",,SQL,GOOGLE DATA STUDIO",",,EXCEL,TABLEAU,SSIS,POWER BI,KPIS,BI,DATABASE...",14,1,2024
1,ACB - Ngân Hàng TMCP Á Châu,Hồ Chí Minh,2024-01-08,CareerBuilder,Business Analyst,1,"DATABASE,PROGRAMMING,COMPUTER SCIENCE,BUSINESS...",,DATABASE,SQL,"DATABASE,SQL,VIETNAMESE,ENGLISH,DATABASE SCHEM...",8,1,2024
2,ADPMN IT SOLUTIONS,Georgia,2024-01-14,Indeed,Database Administrator,1,,,,,,14,1,2024
3,"ADS, Inc.","Austin, TX",2024-01-14,Indeed,Frontend Developer,1,"COLOR,WEB DEVELOPMENT,WEB DESIGN,MOBILE APPS,D...","WEB DEVELOPMENT,REACT",REACT,"SQL,PYTHON,JAVASCRIPT,HTML,CSS,JS","ADOBE ILLUSTRATOR,WEB DEVELOPMENT,SQL,WEB DESI...",14,1,2024
4,"ADS, Inc.","Austin, TX",2024-01-14,Indeed,Mobile Developer,1,"COLOR,WEB DEVELOPMENT,WEB DESIGN,MOBILE APPS,D...","WEB DEVELOPMENT,REACT",REACT,"SQL,PYTHON,JAVASCRIPT,HTML,CSS,JS","ADOBE ILLUSTRATOR,WEB DEVELOPMENT,SQL,WEB DESI...",14,1,2024


## Import data to neo4j
### Support function 

In [20]:

def createNewNode(nodeName, propertiesList):
    properties = {k: v for k, v in propertiesList.items()}
    new_node = Node(nodeName, **properties)
    graph.create(new_node)
    return new_node



def getOrCreateNode(nodeName, attribute, attributeValue):
    cypher_query = ""
    if (pd.isna(attributeValue)): return None

    if (type(attributeValue) == str): 
        if(len(attributeValue) < 2): return None
        attributeValue = attributeValue.replace("\"", "'")
        cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attribute}=\"{attributeValue}\" RETURN n"
    else:
        cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attribute}={attributeValue} RETURN n"

    result = graph.evaluate(cypher_query)
    if (result is None):
        newNode = Node(nodeName, **{attribute: attributeValue})
        result = graph.create(newNode)
        result = newNode
    

    return result



def getOrCreateNodeMultiAttr(nodeName, attrChecker, AttrCheckerValue, propertiesList):
    # Construct the WHERE clause for the Cypher query
    if (pd.isna(AttrCheckerValue)): return None
    cypher_query =""
    if (type(AttrCheckerValue) == str): 
        if len(AttrCheckerValue) < 2:  return None
        AttrCheckerValue = AttrCheckerValue.replace("\"", "'")
        cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attrChecker}=\"{AttrCheckerValue}\" RETURN n"
    else:
        cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attrChecker}={AttrCheckerValue} RETURN n"

    result = graph.evaluate(cypher_query)

    if result is None:
        # Dynamically set properties using keyword arguments
        properties = {k: v for k, v in propertiesList.items()}
        new_node = Node(nodeName, **properties)
        graph.create(new_node)
        return new_node

    return result

def makeRelationship(sourceNode, relation, targetNode):
    if (sourceNode is None or targetNode is None): return
    graph.merge(Relationship(sourceNode,relation, targetNode))




In [10]:
# def getOrCreatetTimeNode(date, month, year):
#     query = f"MATCH (y:Year {{ year: {year} }})<-[:belong_to_year]-(m:Month {{ month: {month} }})\
#     <-[:belong_to_month]-(d:Date {{ day: {date} }})\
#     RETURN d, m, y"

#     result =  graph.run(query)
    
#     if (result is None):
#         Date = getOrCreateNode("Date", "day", date)
#         Month = getOrCreateNode("Month", "month", month)
#         Year = getOrCreateNode("Year", "year", year)
#         graph.merge(Relationship(Date, "belong_to_month", Month))
#         graph.merge(Relationship(Month, "belong_to_year", Year))
#         return [Date, Month, Year]

#     # Extracting nodes from the result
#     nodes = [record for record in result]
    
#     return nodes


def getOrCreatetMonthNode(month, year):
    
    cQuery = f"MATCH (m: Month {{month:{month}}})-[:part_of]->(y: Year {{year:{year}}})  RETURN m"
    result = graph.evaluate(cQuery)

    if result is None:
        Year = getOrCreateNode("Year", "year", year)
        Month = Node("Month", month=month)
        graph.create(Month)
        graph.merge(Relationship(Month, "part_of", Year))
        return Month

    return result



def getOrCreatetDateNode(date, month, year):

    # Construct the WHERE clause for the Cypher query
    cypher_query = f"MATCH (n: Date {{day:{date}}})-[:part_of]->(m: Month {{month:{month}}})-[:part_of]->(y: Year {{year:{year}}})  RETURN n"
    
    result = graph.evaluate(cypher_query)

    if result is None:
        Month = getOrCreatetMonthNode(month, year)
        Date = Node("Date", day=date)
        graph.create(Date)
        graph.merge(Relationship(Date, "part_of", Month))
        return Date

    return result


#### Testing functions --------------------------------

In [11]:
# getOrCreateNodeMultiAttr("Web", "name", "hcmus", {"name": "hcmus", "url": "https://www.hcmus.com"})
# getOrCreateNodeMultiAttr("Web", "name", "hcmus", {"name": "hcmus", "url": "https://www.hcmus.com"})
# getOrCreateNodeMultiAttr("Web", "name", "gover", {"name": "gover", "url": "https://www.hcmus.com"})

#getOrCreateNode("wibu", "name", "itadori's su")
## graph.delete_all()
# result = getOrCreatetDateNode(14,1,1999)
# result = getOrCreatetDateNode(14,2,1999)
# result = getOrCreatetDateNode(24,1,1999)

# executeQuery("MATCH(n) return n limit 25")


# Import Course data

## Nguyên tắc
Name,Link,Rating,Enroll,Instructor,Time,Level,skillrequirement,SkillWillLearn,Description,SkillGain,Subject,Organization,Price,Subtitle,Knowledge,Tool,Framework,Platform,ProgramingLanguage,Practice Task,webname,weburl,NeedKnowledge,NeedPlatform,NeedFramework,NeedProgrammingLanguage,NeedTool

### Chia thành 3 nhóm: Fact, Competency, Course related. Và gồm 1 + 5 + 6  = 12 thực thể liên kết  với nhau
**Fact**: Bao gồm FactCourse (Enroll, rating) \
***Fact Relationship***:
- Fact - Course: belong_to_course
- Fact - Competency: teach_competency
- Fact - Website: post_on
- Fact - Ozganization: belong_to

**Course**: Bao gồm Course(name, link, description, subject, price), Instructor(name), <br> Website(name, url), Level(level), Organization(name), Subtitle(language) \
***Fact Relationship***:
- Course - Instructor: teach_by
- Course - Competency: teach_competency
- Course - Level: have_level
- Course - Subtitle: have_subtitle

**Competency**: Bao gồm Knowledge (knowledge), ProgrammingLanguage(programmingLanguage),<br> Platform(platform), Framework(framework), Tool(tool) \
***Competency Relationship***:
- Knowledge - ProgrammingLanguage, Platform, Framework, Tool: relate_to
- ProgramingLanguage - Framework: have_framework
- ProgrammingLanguage - Tool: have_tool
- Platform - Framework: deploy_on

In [12]:

def competencyProcess(sourceNode, competency, competencyValue, relationName):

    if (pd.isna(competencyValue) or len(competencyValue) < 1): return

    nodeName = competency
    nodeAttr = competency[:1].lower() + competency[1:]
    
    for comp in competencyValue.split(','):
        if (len(comp) < 2): continue
        compe = getOrCreateNode(nodeName, nodeAttr, comp.strip().upper())
        makeRelationship(sourceNode, relationName, compe)

def processAllCompetency(Knowledge, ProgrammingLanguage, Framework, Platform, Tool):
    if (pd.isna(ProgrammingLanguage) or len(ProgrammingLanguage) < 1): return

    for program in ProgrammingLanguage.split(','):
        if (len(program) < 2): continue

        Program = getOrCreateNode("ProgrammingLanguage", "programmingLanguage", program.strip().upper())
        competencyProcess(Program, "Knowledge", Knowledge, "relate_to_knowledge")
        competencyProcess(Program, "Framework", Framework, "have_framework")
        competencyProcess(Program, "Tool", Tool, "use_tool") 

    if (pd.isna(Framework) or len(Framework) < 1): return

    for frame in Framework.split(','):
        if (len(frame) < 2): continue
        frameNode = getOrCreateNode("Framework", "framework", frame)
        competencyProcess(frameNode, "Platform", Platform, "deploy_to_platform")
        competencyProcess(frameNode, "Knowledge", Knowledge, "relate_to_framework")

In [13]:






def importCourseData(courseData):
    
    mileStone = len(courseData)
    i = 0
    for index, row in courseData.iterrows():

        # create fact group
        if (pd.isna(row['Enroll'])): continue
        
        FactCourse = createNewNode("FactCourse", {"enroll": row['Enroll'], "rating": row['Rating']})
        
        # create compentency 
        #(teached competency)
        competencyProcess(FactCourse, "Knowledge", row['Knowledge'], "teach_knowledge" )
        competencyProcess(FactCourse, "ProgrammingLanguage", row['ProgramingLanguage'], "teach_programmingLanguage" )
        competencyProcess(FactCourse, "Framework", row['Framework'], "teach_framework" )
        competencyProcess(FactCourse, "Platform", row['Platform'], "teach_platform" )
        competencyProcess(FactCourse, "Tool", row['Tool'], "teach_tool")

        processAllCompetency(row['Knowledge'], row['ProgramingLanguage'], row['Framework'], row['Platform'], row['Tool'])


        # Create course related group 
        courseRow = {"name": row['Name'], "description": row['Description'], "link": row['Link'], "subject": row['Subject'],
                      "price": row['Price'], "skillWillLearn": row['SkillWillLearn'], 'practiceTask': row['Practice Task']}
        
        Course = getOrCreateNodeMultiAttr("Course", "link", row['Link'], courseRow)
        Organization = getOrCreateNode("Organization", "name", row['Organization'])
        Level = getOrCreateNode("Level", "level", row['Level'])
        Instructor = getOrCreateNode("Instructor", "name", row['Instructor'])
        Website = getOrCreateNodeMultiAttr("Website", "name", row['webname'], {"name": row['webname'], "url": row['weburl']})
        if not pd.isna(row['Subtitle']):
            for sub in row['Subtitle'].split(","):
                if (len(sub) < 2): continue
                Subtitle = getOrCreateNode("Subtitle", "language", sub.strip().upper())
                makeRelationship(Course, "have_subtitle", Subtitle)

        #(required competency (Pre-skill need to learn))
        competencyProcess(Course, "Knowledge", row['NeedKnowledge'], "nead_knowledge" )
        competencyProcess(Course, "ProgrammingLanguage", row['NeedProgrammingLanguage'], "need_programmingLanguage" )
        competencyProcess(Course, "Framework", row['NeedFramework'], "need_framework" )
        competencyProcess(Course, "Platform", row['NeedPlatform'], "need_platform" )
        competencyProcess(Course, "Tool", row['NeedTool'], "need_tool")

        makeRelationship(Course, "have_level", Level)
        makeRelationship(Course, "teach_by", Instructor)

        makeRelationship(FactCourse, "belong_to", Organization)
        makeRelationship(FactCourse, "post_on", Website)
        makeRelationship(FactCourse, "belong_to_course", Course)
        i = i + 1
        print(f"Loading process {i/mileStone * 100} %", end='\r')

    print("\nDone!")

        




In [21]:

graph.delete_all()
importCourseData(courseDataDf)


Loading process 100.0 %2576687117 %%%
Done!


In [22]:


executeQuery("MATCH(n) return n limit 25")


[Record({'n': Node('FactCourse', enroll=529, rating=4.8)}),
 Record({'n': Node('Knowledge', knowledge='PROTOCOLS WEB BROWSERS')}),
 Record({'n': Node('Knowledge', knowledge='REGULAR EXPRESSIONS')}),
 Record({'n': Node('Course', description='', link='https://www.coursera.org/learn/python-network-data', name='Using Python to Access Web Data', practiceTask='extract data, retrieve data, retrieve documents and web apps', price=0.0, skillWillLearn='Use regular expressions to extract data from strings. Understand the protocols web browsers use to retrieve documents and web apps. Retrieve data from websites and APIs using Python. Work with XML (eXtensible Markup Language) data', subject='Computer Science')}),
 Record({'n': Node('Instructor', name='Charles Russell Severance')}),
 Record({'n': Node('FactCourse', enroll=137, rating=4.5)}),
 Record({'n': Node('Knowledge', knowledge='DATAFRAME')}),
 Record({'n': Node('Knowledge', knowledge='CSV FILES')}),
 Record({'n': Node('Framework', framework='

# Import job data

In [23]:
def importJobData(df):
    # Iterate through rows and create nodes and relationships
    mileStone = len(df)
    i = 0
    for index, row in df.iterrows():
            
        Organization = getOrCreateNode("Organization", "name", row['companyName'])
        Location = getOrCreateNode( "Location", "location", row['location'])
      
        webName = row['website']
        webUrl =  "https://www." + webName + ".com"
        Website = getOrCreateNodeMultiAttr("Website", "name", webName, {"name": webName, "url": webUrl})

        # Create Career node or retrieve existing one
        Career = getOrCreateNode( "Career", "name", row['jobType'])
    
        # Create FactJobPosting node or retrieve existing one
        FactJobPosting = Node("FactJobPosting", totalJobPost=row['jobCount'])
        graph.create(FactJobPosting)

        # Create Date node or retrieve existing one
        timePost = getOrCreatetDateNode(row['date'],row['month'], row['year'])
        
        makeRelationship(FactJobPosting, "recruited_by", Organization)
        makeRelationship(FactJobPosting, "public_on", Website)
        makeRelationship(FactJobPosting, "belong_to_career", Career)
        makeRelationship(FactJobPosting, "located_at", Location)
        makeRelationship(FactJobPosting, "post_at_time", timePost)
        
        competencyProcess(FactJobPosting, "Knowledge", row['Knowledge'], "require_knowledge" )
        competencyProcess(FactJobPosting, "ProgrammingLanguage", row['ProgrammingLanguage'], "require_programmingLanguage" )
        competencyProcess(FactJobPosting, "Framework", row['Framework'], "require_framework" )
        competencyProcess(FactJobPosting, "Platform", row['Platform'], "require_platform" )
        competencyProcess(FactJobPosting, "Tool", row['Tool'], "require_tool")

        i = i + 1
        print(f"Loading process {i/mileStone * 100} %", end='\r')
        


In [24]:
importJobData(jobDataDf)

In [ ]:
len(jobDataDf)

1076

## Some query to analyze
- query1: match(f:FactJobPosting)-[:require_programmingLanguage]->(p:ProgrammingLanguage) return p.programmingLanguage as `Programming Language`, count(f) as `number of job`
- query2: match(f:FactJobPosting)-[:belong_to_career]->(p:Career) return p.name as Career, count(f) as `number of job`
- query3: match(f:FactJobPosting)-[:post_at_time]->(d:Date)-[:part_of]->(m:Month)-[:part_of]->(y:Year)return m.month as Month,y.year as Year, count(f) as `number of job`